## Multiprocessing and Class

$\color{red}{\text{why Multiprocessing not working on Windows and how to make it work}}$ 

### Define a class

In [1]:
class Listener:
    def __init__(self,opla_job):
        self.opla_job=opla_job
        self.lst=[]
    def test(self):
        for i in range(0,3):
            self.lst.append(i)

In [2]:
import multiprocessing

In [3]:
active_lst=['listener1','listener2']
all_active_process = {}

In [4]:
for opla_job in active_lst:
    print('Starting to create a new process:', opla_job)
    instance = Listener(opla_job)
    print(instance)
    p = multiprocessing.Process(target=instance.test, name=opla_job)
    p.start()
    all_active_process[opla_job] = p
    print(all_active_process)

Starting to create a new process: listener1
{'listener1': <Process name='listener1' pid=20484 parent=3364 started>}
Starting to create a new process: listener2
{'listener1': <Process name='listener1' pid=20484 parent=3364 stopped exitcode=1>, 'listener2': <Process name='listener2' pid=18984 parent=3364 started>}


In [5]:
import time
time.sleep(1)
all_active_process

{'listener1': <Process name='listener1' pid=20484 parent=3364 stopped exitcode=1>,
 'listener2': <Process name='listener2' pid=18984 parent=3364 stopped exitcode=1>}

### note exitcode = 1 above

### To fix: add ()

In [6]:
for opla_job in active_lst:
    print('Starting to create a new process:', opla_job)
    instance = Listener(opla_job)
    print(instance)
    p = multiprocessing.Process(target=instance.test(), name=opla_job)
    p.start()
    all_active_process[opla_job] = p
    print(all_active_process)

Starting to create a new process: listener1
{'listener1': <Process name='listener1' pid=12840 parent=3364 started>, 'listener2': <Process name='listener2' pid=18984 parent=3364 stopped exitcode=1>}
Starting to create a new process: listener2
{'listener1': <Process name='listener1' pid=12840 parent=3364 stopped exitcode=0>, 'listener2': <Process name='listener2' pid=2828 parent=3364 started>}


In [7]:
import time
time.sleep(1)
all_active_process

{'listener1': <Process name='listener1' pid=12840 parent=3364 stopped exitcode=0>,
 'listener2': <Process name='listener2' pid=2828 parent=3364 stopped exitcode=0>}

### note exitcode = 0 above

### what if use import

In [1]:
from Listener import Listener
import multiprocessing

In [2]:
active_lst=['listener1','listener2']
all_active_process = {}

In [3]:
for opla_job in active_lst:
    print('Starting to create a new process:', opla_job)
    instance = Listener(opla_job)
    print(instance)
    p = multiprocessing.Process(target=instance.test, name=opla_job)
    p.start()
    all_active_process[opla_job] = p
    print(all_active_process)

Starting to create a new process: listener1
{'listener1': <Process name='listener1' pid=20336 parent=9184 started>}
Starting to create a new process: listener2
{'listener1': <Process name='listener1' pid=20336 parent=9184 stopped exitcode=0>, 'listener2': <Process name='listener2' pid=10840 parent=9184 started>}


In [4]:
import time
time.sleep(1)
all_active_process

{'listener1': <Process name='listener1' pid=20336 parent=9184 stopped exitcode=0>,
 'listener2': <Process name='listener2' pid=10840 parent=9184 stopped exitcode=0>}

### note exitcode = 0 above, but no need to add ()

In [2]:
listener1=Listener('yy')
listener2=Listener('qq')

In [3]:
listener1.test()

In [4]:
listener1.lst

[0, 1, 2]

In [5]:
listener2.test

<bound method Listener.test of <Listener.Listener object at 0x0000026F9BF71B20>>

In [6]:
listener2.lst

[]

In [7]:
listener2.test()

In [8]:
listener2.lst

[0, 1, 2]

# Root Problem I guess

In [3]:
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f, args=('bob',))
    p.start()
    p.join()

In [4]:
p

<Process name='Process-1' pid=16000 parent=20956 stopped exitcode=1>

### $\color{red}{\text{Multiprocessing not working on Windows !!!}}$ 

**The Windows API only supports the spawn method of process creation.** In POSIX (except macOS), the default is the fork method, for which the child inherits the interactive main module of the parent. If you switch to the spawn method in Linux via **multiprocessing.set_start_method('spawn')**, you'll see the same error. 

**multiprocessing is one package where it's necessary in Windows to test examples using a script.** This is implied in the guidelines when it says to "[m]ake sure that the main module can be safely imported by a new Python interpreter without causing unintended side effects (such a starting a new process)". It's the fact the main module has to be importable by child processes that matters in this case. The behavior is noted with an example at the end of the introduction:

    Note
    Functionality within this package requires that the __main__ module
    be importable by the children. This is covered in Programming 
    guidelines however it is worth pointing out here. This means that 
    some examples, such as the multiprocessing.pool.Pool examples will 
    not work in the interactive interpreter. For example: ...

If it were up to me this note would be at the beginning of the introduction, where everyone would certainly see it. As is, the reader is expected to at least scan over the entire introduction.

* **macOS defaults to the spawn method**

In [7]:
from multiprocessing import Process

def f(name):
    print('hello', name)

if __name__ == '__main__':
    p = Process(target=f('bob'))
    p.start()
    p.join()

hello bob


In [8]:
p

<Process name='Process-4' pid=14664 parent=21148 stopped exitcode=0>

### $\color{red}{\text{Although exitcode = 0 above, but it is not multiprocessing !!! Check below!!!}}$ 

In [1]:
import multiprocessing
import time
 
def task():
    print('Sleeping for 2 seconds')
    time.sleep(2)
    print('Finished sleeping')

if __name__ == "__main__":
    start_time = time.time()
 
    # Creates two processes
    p1 = multiprocessing.Process(target=task())
    p2 = multiprocessing.Process(target=task())
 
    # Starts both processes
    p1.start()
    p2.start()
    
    # p1.join()
    # p2.join()
 
    finish_time = time.time()
    
    print(f"Program finished in {finish_time-start_time} seconds")

Sleeping for 2 seconds
Finished sleeping
Sleeping for 2 seconds
Finished sleeping
Program finished in 4.269025802612305 seconds


In [2]:
p1

<Process name='Process-1' pid=22404 parent=13508 stopped exitcode=0>

In [3]:
p2

<Process name='Process-2' pid=21608 parent=13508 stopped exitcode=0>

### $\color{red}{\text{Not Multiprocessing!!! Program executed one by one in order!!!}}$ 

In [4]:
def calc_square(numbers):
    for n in numbers:
        print('square ' + str(n*n))

def calc_cube(numbers):
    for n in numbers:
        print('cube ' + str(n*n*n))

if __name__ == "__main__":
    arr = [2,3,8]
    p1 = multiprocessing.Process(target=calc_square(arr))
    p2 = multiprocessing.Process(target=calc_cube(arr))

    p1.start()
    p2.start()

    p1.join()
    p2.join()

    print("Done!")

square 4
square 9
square 64
cube 8
cube 27
cube 512
Done!


In [6]:
p1

<Process name='Process-3' pid=18996 parent=13508 stopped exitcode=0>

In [7]:
p2

<Process name='Process-4' pid=8876 parent=13508 stopped exitcode=0>